Created on Fri Jan 10 11:06:30 2020
<br>
Group 7
<br>
@author: C.L.

<h1>Group 7 - Images sociales<span class="tocSkip"></span>
    
<br>  
<center>Scraping

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Environment" data-toc-modified-id="Environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Environment</a></span><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Parameters</a></span></li></ul></li><li><span><a href="#Airliners" data-toc-modified-id="Airliners-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Airliners</a></span><ul class="toc-item"><li><span><a href="#Functions" data-toc-modified-id="Functions-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Extract-images-info" data-toc-modified-id="Extract-images-info-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Extract images info</a></span></li><li><span><a href="#Retreive-and-save-images" data-toc-modified-id="Retreive-and-save-images-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Retreive and save images</a></span></li></ul></li><li><span><a href="#Google-images" data-toc-modified-id="Google-images-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Google images</a></span><ul class="toc-item"><li><span><a href="#Functions" data-toc-modified-id="Functions-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Extract-info,-retreive-and-save-images" data-toc-modified-id="Extract-info,-retreive-and-save-images-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Extract info, retreive and save images</a></span></li></ul></li></ul></div>

# Introduction

To know before scraping : Airliners images are protected by copyright.

In order to train a model able to recognize aircraft types from the outside, we needed training images.
For this purpose, we scrapped images from Airliners and Google Images. The method for one website is the following:
* Extract relevant web links, and store them into a DataFrame (saved as `CSV`);
* Use these DataFrames to extract and store images.

Running the following code takes a bit of time. Since we already performed this scraping task, you don't need to run it again (that said, you can use the code as a template and adapt it to other websites you might want to srcap).

# Environment
To ensure a proper functioning of this code file, `python 3.6` or later version is required.

Please note that Google Images scraping requires the installation of `ChromeDriver`: https://chromedriver.chromium.org/.

## Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import csv
from PIL import Image
import io
from datetime import datetime, timedelta
import time
import numpy as np
import errno
import os
import urllib.request
import urllib
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

In [2]:
%load_ext watermark
%watermark -p selenium,bs4,PIL

selenium 3.141.0
bs4 4.8.1
PIL 6.2.0


## Parameters

In [ ]:
# Paths
project_path = './../'
scrap_path = project_path + 'Scraping/'
airliners_csv_path = scrap_path + 'Airliners/'
airliners_path = airliners_csv_path + 'data/'
google_path = scrap_path + 'Google_img/'

# Airliners
Boeing : 28482 pages found
<br>
Airbus : 17453 pages found

## Functions

In [ ]:
def create_directories(path: str):
    """Creates directories to store Airbus and Boeing scrapped images"""
    os.makedirs(path + 'Airbus', exist_ok=True)
    os.makedirs(path + 'Boeing', exist_ok=True)


def get_photos_name_page(link: str) -> pd.DataFrame:
    """From a given Airliner link, creates a DataFrame containing, for each image found:
        - link to the image
        - manufacturer
        - aircraft type 
        - a more precise designation

    """

    req = requests.get(link)
    soup = BeautifulSoup(req.text, "lxml")
    AirCrafts = soup.find_all("div", {"class": "ps-v2-results-row"})
    list_DF = []

    for air in AirCrafts:
        try:
            photo = air.find_all("img")[0].attrs['src']
            details_cell = air.find_all(
                "div", {"class": "ps-v2-results-col ps-v2-results-col-aircraft"})[0]
            details_cell = details_cell.find_all(
                "div", {'class': "ps-v2-results-col-content"})[0]
            details_cell = details_cell.find_all(
                "div", {"class": "ps-v2-results-col-content-primary"})[0]
            name_cell = details_cell.find_all(
                "div", {"class": "ps-v2-results-display-detail-no-wrapping"})[1]
            name = name_cell.a.text.strip().replace('/', '-')
            if name.split()[0] in ['Airbus', 'Boeing']:
                list_DF.append(
                    [photo, name.split()[0], name.split()[1].split('-')[0], name])
        except:
            pass

    DF = pd.DataFrame(list_DF, columns=[
                      'Photo', 'Manufacturer', 'Aircraft_type', 'Designation'])

    return(DF)


def create_df_manufacturer(a: int, b: int, man_name: str) -> pd.DataFrame:
    """For a given manufacturer, creates a DataFrame in which each line contains information related to an aircraft 
    image.

    Parameters: 
        a: page number where the search begins
        b: page number where the search ends
        man_name: aircraft manufacturer name

    Out:
        df: DataFrame with retreived information

    """

    l_df = []
    dic_code_man = {'Airbus': '2', 'Boeing': '7'}
    for i in range(a, b):
        print(i)
        link = 'https://www.airliners.net/search?aircraftManufacturer=' + dic_code_man[man_name] + \
               '&photoCategory=23&sortBy=dateAccepted&sortOrder=desc&perPage=36&display=detail&page=' + \
            str(i)
        l_df.append(get_photos_name_page(link))
    df = pd.concat(l_df, ignore_index=True)

    return df


def create_df_passengers(df: pd.DataFrame, manufacturer: str, dic_type: dict):
    """Creates a new DataFrame with only rows matching with commercial aircraft and adds a column
    similar to the index.

    Parameters : 
        df: DataFrame on which the selection must be proceed
        manufacturer: company name ('Airbus' or 'Boeing')
        dic_type: keys are companies names and values are lists of the commercial aircraft of the manufacturer 

    Out :
        df_passengers: modified DataFrame 

    """

    df_passengers = df[df['Aircraft_type'].isin(dic_type[manufacturer])]

    return df_passengers.assign(Number=df_passengers.index)


def get_nb_imgs_per_type(df: pd.DataFrame, nb: int, a: int) -> pd.DataFrame:
    """
    Parameters:
        df: DataFrame with only aircraft types of interest
        nb: number of images to create for each aircraft type
        a: lag

    Out:
        df_nb: DataFrame containing the number of images per aircraft type. 

    """

    # nb :
    ix = []
    for el in df['Aircraft_type'].unique().tolist():
        df_el = df[df['Aircraft_type'] == el].copy()
        df_el.reset_index(drop=True, inplace=True)
        df_el = df_el.loc[a: a + min(nb-1, len(df_el))]
        ix.extend(df_el['Number'].tolist())
    df_nb = df[df['Number'].isin(ix)]

    return df_nb


def get_from_link_and_write(df: pd.DataFrame, man: str, path: str):
    """Retreives and saves images.

    Parameters:
        df: DataFrame with links of images to scrap
        man: aircraft manufacturer
        path: where to save images

    """

    for el in df['Aircraft_type'].unique().tolist():
        df_type = df[df['Aircraft_type'] == el]
        fold_name = path + man + '/' + el
        os.makedirs(fold_name, exist_ok=True)
        my_rows = zip(df_type['Number'], df_type['Photo'])
        for (num, photo) in my_rows:
            img_data = requests.get(photo).content
            try:
                i = Image.open(io.BytesIO(img_data))
                i.save(fold_name + '/' + man +
                       '_' + str(num) + '_' + el + '.jpg')
            except:
                print(el, num)
                pass

## Extract images info

In [ ]:
# Create Airbus and Boeing directories where to save images
create_directories(airliners_path)

Running the following code blocks takes a bit of time.

In [ ]:
# Scrap Airbus images
df_boeing_airliners = create_df_manufacturer(
    1, 500, 'Airbus')  # retreive info
df_boeing_airliners.to_csv(
    scrap_path + 'Airbus_Airliners.csv', sep=';', index=False)  # save to csv
len(df_airbus_airliners)

In [ ]:
# Scrap Boeing images
df_boeing_airliners = create_df_manufacturer(
    1, 500, 'Boeing')  # retreive info
df_boeing_airliners.to_csv(
    scrap_path + 'Boeing_Airliners.csv', sep=';', index=False)  # save to csv
len(df_airbus_airliners)

In [ ]:
# Code to retreive DataFrames from csv if need be
df_airbus = pd.read_csv(scrap_path + 'Airbus_Airliners.csv', sep=';')
df_boeing = pd.read_csv(scrap_path + 'Boeing_Airliners.csv', sep=';')

In [ ]:
# Dict with aircraft types for each manufacturer
dic_type = {'Airbus': ['A220',
                       'A300B4',
                       'A310',
                       'A318',
                       'A319',
                       'A320',
                       'A321',
                       'A330',
                       'A340',
                       'A350',
                       'A380'
                       ],
            'Boeing': ['717',
                       '727',
                       '737',
                       '747',
                       '757',
                       '767',
                       '777',
                       '787'
                       ]
            }

In [ ]:
# Creates new DataFrames with only rows matching with commercial aircraft
df_airbus_passengers = create_df_passengers(df_airbus, 'Airbus', dic_type)
df_boeing_passengers = create_df_passengers(df_boeing, 'Boeing', dic_type)

In [ ]:
# Print number of elements per aircraft type
for el in dic_type['Airbus']:
    print(el, len(df_airbus[df_airbus['Aircraft_type'] == el]))

print('\n')

for el in dic_type['Boeing']:
    print(el, len(df_boeing[df_boeing['Aircraft_type'] == el]))

## Retreive and save images

In [ ]:
# Airbus
df_airbus_limited = get_nb_imgs_per_type(df_airbus_passengers, 500, 0)
print(len(df_airbus_100))

start = time.time()
get_from_link_and_write(df_airbus_limited, 'Airbus', airliners_path)
print(time.time()-start)


In [ ]:
# Boeing
df_boeing_limited = get_nb_imgs_per_type(df_boeing_passengers, 500, 0)
get_from_link_and_write(df_boeing_limited, 'Boeing', airliners_path)

# Google images

## Functions

In [ ]:
def google_img_by_aircraft_type(aircraft_type: str, dic_df_types: dict):
    """ Updates a dictionary with a DataFrame giving links to the pictures found on the
    first Google Images page for a given request.

    Parameters :
        aircraft_type: request for Google Images, e.g.: 'Airbus A380'.
        dic_df_types: keys = aircraft_type; values = DataFrames.

    """

    # Use of &tbs=sur:fc in the link to download only free_to_use images:
    link = 'https://www.google.com/search?q=' + \
        aircraft_type + "&tbm=isch&tbs=sur:fc"
    df_images = pd.DataFrame(columns=["link", "request", "source", "source_2"])
    driver.get(link)
    time.sleep(5)

    blocks_images = driver.find_elements_by_xpath(
        "//div[contains(@class,'rg_bx rg_di rg_el ivg-i')]")
    for bi in blocks_images:

        site = bi.find_elements_by_xpath(".//span")
        if site:
            site = site[0].get_attribute("innerHTML")

        source_1 = bi.find_elements_by_xpath(".//img")
        if source_1:
            source_1 = source_1[0].get_attribute("data-src")

        source_2 = bi.find_elements_by_xpath("./a")
        if source_2:
            source_2 = source_2[0].get_attribute("href")

        row = pd.Series([site, aircraft_type, source_1,
                         source_2], index=df_images.columns)
        df_images = df_images.append(row, ignore_index=True, sort=False)

    dic_df_types[aircraft_type] = df_images


def get_from_source_and_write(request: str, dic_df_types: dict, path: str):
    """Retreives and saves images.

    Parameters:
        request: manufacturer name + white space + aircraft-type (e.g.: 'Airbus A320')
        dic_df_types: 
        path: path to the Google subdirectory

    """

    print(request)
    df = dic_df_types[request]
    print(len(df))
    df = df[df['source'].notnull()]
    df.reset_index(inplace=True)
    df = df.assign(Number=df.index)
    print(len(df))
    request = request.split()
    fold_name = path + request[0] + '/' + request[1]
    os.makedirs(fold_name, exist_ok=True)
    my_rows = zip(df['Number'], df['source'])
    for (num, source) in my_rows:
        img_data = requests.get(source).content
        try:
            i = Image.open(io.BytesIO(img_data))
            i.save(fold_name + '/' +
                   request[0] + '_' + request[1] + '_' + str(num) + '.jpg')
        except:
            pass

In [ ]:
# Set Chrome Driver and create Airbus and Boeing directories where to save images
driver = webdriver.Chrome("chromedriver.exe")
os.makedirs(google_path, exist_ok=True)

# Init
dic_df_types = {}

## Extract info, retreive and save images

Running the following code blocks takes a bit of time.

In [ ]:
# Fill the dict with links to the pictures to scrap
for key in dic_type.keys():
    print(key)
    for aircraft in dic_type[key]:
        print(aircraft)
        google_img_by_aircraft_type(key + ' ' + aircraft, dic_df_types)

In [ ]:
# Retreive and save images
for key in dic_type.keys():
    for aircraft in dic_type[key]:
        get_from_source_and_write(
            key + ' ' + aircraft, dic_df_types, google_path)

In [ ]:
# Save DataFrames to csv
for key, val in dic_df_types.items():
    val.to_csv(google_path + "Google_{}.csv".format(str(key)),
               sep=';', index=False)

# Save keys to txt file
with open(google_path + "keys.txt", "w") as f:
    f.write(str(list(dic_df_types.keys())))

# Read data from keys
with open(google_path + "keys.txt", "r") as f:
    keys = eval(f.read())
    dic = {}
    for key in keys:
        dic[key] = pd.read_csv(
            google_path + "Google_{}.csv".format(str(key)), sep=';')
    return dic